In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk

INSTANCES_PATH = Path('/home/joasiee/Documents/instances/synthetic/scans/')

In [2]:
def read_image(path: Path):
    image = sitk.ReadImage(str(path.resolve()))
    return image

In [64]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook_connected"
import numpy as np

X, Y, Z = np.mgrid[0:30, 0:30, 0:30]
values = sitk.GetArrayFromImage(read_image(INSTANCES_PATH / '01_Fixed.mhd'))

layout = go.Layout(
    scene=dict(
        xaxis = go.layout.scene.XAxis(showticklabels=False, title=''),
        yaxis = go.layout.scene.YAxis(showticklabels=False, title=''),
        zaxis = go.layout.scene.ZAxis(showticklabels=False, title=''),
    )
)

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=values.flatten(),
    showscale=False,
    isomin=100,
    isomax=200,
    opacity=0.1,
    colorscale='gray',
    surface_count=3,
    ), layout=layout)

fig.write_image("img/synthetic_1_fixed.pdf", scale=5, width=1000, height=1000)